In [2]:
import pandas as pd
import numpy as np

all_feature = pd.read_csv('/kaggle/input/abc-feature-labels/all_features(with jerk).csv')
all_label = pd.read_csv('/kaggle/input/abc-feature-labels/all_labels(with jerk).csv')

In [3]:
all_feature = all_feature.to_numpy()
print(all_feature.shape)
print(all_label.shape)
all_label = all_label.to_numpy()
all_label = all_label.T.flatten()
print(all_label.shape)

from xgboost import XGBClassifier

(9716, 679)
(9716, 1)
(9716,)


In [4]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [4]:
# Define XGBC
model = XGBClassifier(random_state=42)

# Create Pipeline
steps = list()
steps.append(('scaler', MinMaxScaler()))
steps.append(('classifier', model))
pipeline = Pipeline(steps=steps)

# Define Parameters

params= {
    'classifier__learning_rate': [0.3, 0.1, 0.01, 0.001],
    'classifier__max_depth':[1, 3, 5, 10, 15, 20],
    'classifier__min_child_weight': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'classifier__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'classifier__colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'classifier__n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]   
}

In [5]:
#Random search for ideal n_estimators
print(f'Searching for the best value of parameter for {model}...')
classifier = RandomizedSearchCV(pipeline, params, cv=5, n_jobs=-1, scoring='f1_macro').fit(all_feature, all_label)

Searching for the best value of parameter for XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)...


In [6]:
print(classifier.cv_results_)
print(classifier.best_params_)
print(classifier.best_score_)

{'mean_fit_time': array([ 335.01836624,  352.71119857,  842.03974824,  264.60356288,
        763.94471469,  361.27046428,  658.87858138,  274.59868097,
        637.13084025, 2012.99875693]), 'std_fit_time': array([  5.43495234,  10.85406646,  10.31074642,   4.15882746,
         8.91261015,   4.68474744,   4.92440637,   2.29613875,
        10.97854542, 251.33286691]), 'mean_score_time': array([0.19079752, 0.18921905, 0.46103029, 0.16441598, 0.40505953,
       0.15457807, 0.37902775, 0.12577448, 0.32993107, 1.25540094]), 'std_score_time': array([0.02539847, 0.00650911, 0.04773975, 0.00254092, 0.0052013 ,
       0.00367312, 0.05661182, 0.00070699, 0.00324193, 0.15271334]), 'param_classifier__subsample': masked_array(data=[0.6, 1.0, 0.5, 0.8, 0.5, 0.9, 0.9, 0.5, 0.7, 1.0],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_classifier__n_estimators': masked_array(data=[500, 100

In [6]:
#{'classifier__subsample': 0.5, 'classifier__n_estimators': 900, 'classifier__min_child_weight': 2, 'classifier__max_depth': 5, 'classifier__learning_rate': 0.1, 'classifier__colsample_bytree': 0.7}
#0.5642738288440016
#model_final = XGBClassifier(C=10, class_weight=None,solver='saga', max_iter=500, random_state=42)
model_final = XGBClassifier(subsample= 0.5, n_estimators= 900, min_child_weight=2,\
                            max_depth = 5, learning_rate= 0.1, colsample_bytree = 0.7)

# Split train-test data
f_train,f_test,l_train,l_test = train_test_split(all_feature, all_label, test_size = 0.2,\
                                                 random_state=42, shuffle = True)

# Scale data
scaler = MinMaxScaler()
scaler.fit(f_train)
f_train = scaler.transform(f_train)
f_test = scaler.transform(f_test)


# Fit + Train data
model_final.fit(f_train,l_train)
prediction = model_final.predict(f_test)

# Evaluate
print(classification_report(l_test,prediction))

              precision    recall  f1-score   support

           0       0.85      0.93      0.89       589
           1       0.68      0.62      0.65        34
           2       0.98      0.92      0.95       171
           3       0.77      0.55      0.64        49
           4       0.85      0.77      0.81       318
           5       0.90      0.68      0.77        53
           6       0.76      0.73      0.75        52
           7       0.91      0.97      0.94       551
           8       0.98      0.87      0.92       127

    accuracy                           0.88      1944
   macro avg       0.85      0.78      0.81      1944
weighted avg       0.88      0.88      0.88      1944

